# Введение в анализ данных (первый семестр)
# Задание №4

*** Конопаткин Олег, BD11 ***

In [7]:
import requests
import json
import html
import urllib
from selenium import webdriver

In [73]:
from selenium import webdriver
from time import sleep

def get_books_urls(driver, url, t_sleep=1):
    driver.get(url)
    sleep(t_sleep)
    
    while True:
        for i in range(3):
            driver.execute_script("window.scrollTo(0, 10000000);")
            sleep(3)
        try:
            elem = driver.find_element_by_class_name('js__show-more-cards')
            elem.click()
        except:
            break
    
    urls = []
    
    items = driver.find_elements_by_class_name("product-card__img")
    
    for item in items:
        url = item.get_attribute("href")
        urls.append(url[:url.rfind('/')])
    
    print(len(items))
    
    return urls

In [74]:
# собираем страницы с книгами

authors = ['Фрай М.', 'Хантер Э.', 'Емец Д.']
card_urls = []
base_url = "https://www.chitai-gorod.ru/search/result.php?q={}&type=author"

with webdriver.Chrome(executable_path='C:\\Anaconda3\\chromedriver.exe') as driver:
    for author in authors:
        url = base_url.format(urllib.parse.quote(author, encoding='windows-1251'))
        cur_urls = get_books_urls(driver, url)
        card_urls.extend(cur_urls)

153
97
191


In [79]:
card_ids = [int(url[url.rfind('/') + 1:]) for url in card_urls]
print(card_ids)

[244296, 245728, 246246, 246247, 246533, 246749, 246913, 246914, 247198, 247199, 247200, 247406, 248903, 253620, 256186, 259037, 259751, 259790, 266507, 266508, 268263, 270861, 271107, 271108, 272859, 274724, 274979, 280462, 291153, 292170, 293276, 296766, 300551, 300799, 301879, 302904, 305522, 305523, 309851, 310895, 310896, 312706, 323381, 324455, 350170, 353710, 354139, 354742, 358088, 358727, 361307, 361874, 364589, 367135, 376401, 377037, 379158, 389640, 389641, 392051, 392052, 400412, 411548, 411549, 411550, 411551, 411552, 411553, 411625, 411626, 411627, 411628, 411629, 411682, 411683, 411692, 411693, 411732, 411733, 411750, 442981, 442982, 471783, 710328, 723716, 765102, 779618, 784979, 789215, 790009, 793750, 810832, 812827, 816107, 818391, 820823, 822874, 824732, 825450, 828740, 831871, 839265, 843835, 847078, 848994, 850216, 855443, 857607, 860114, 861223, 867637, 868652, 870661, 870662, 870663, 872961, 875812, 882380, 886599, 888578, 894293, 898955, 901916, 906001, 915767,

In [131]:
'''def get_text(soup, tag, class_):
    text = soup.find(tag, class_=class_)
    if text is not None:
        return text.text.strip()
    return text

def extract_book_info(card_id):
    print(card_id)
    card_url = 'https://www.chitai-gorod.ru/catalog/book/{}'.format(card_id)
    while True:
        card_html = requests.get(card_url).text
        if card_html.find('Request unsuccessful') == -1:
            break
        print(card_html)
        sleep(5) # не спасает
    soup = BeautifulSoup(card_html, 'html.parser')
    
    titles = list(map(lambda f: f.text.strip(), soup.find_all('div', class_='product-prop__title')))
    values = list(map(lambda f: f.text.strip(), soup.find_all('div', class_='product-prop__value')))
    result = {t: v for (t, v) in zip(titles, values)}
    
    result['ID карточки'] = card_id
    
    result['Название'] = get_text(soup, 'h1', 'product__title')
    result['Автор'] = get_text(soup, 'a', 'product__author')
    
    rating = get_text(soup, 'div', 'rating')
    print(rating)
    if rating is None:
        print(card_html)
    result['Рейтинг'] = float(rating[:rating.find('(')])
    result['Голоса'] = int(rating[rating.find('(') + 1:rating.find(')')])
    
    result['Цена'] = get_text(soup, 'div', 'price')
    
    return result'''

In [205]:
def get_text(soup, class_name):
    text = driver.find_element_by_class_name(class_name)
    if text is not None:
        return text.text.strip()
    return text

def extract_book_info(driver, card_id):
    print('{} from {};'.format(card_ids.index(card_id) + 1, len(card_ids)), end = ' ')
    card_url = 'https://www.chitai-gorod.ru/catalog/book/{}'.format(card_id)
    driver.get(card_url)
    
    titles = driver.find_elements_by_class_name('product-prop__title')
    titles = list(map(lambda f: f.text.strip(), titles))
    values = driver.find_elements_by_class_name('product-prop__value')
    values = list(map(lambda f: f.text.strip(), values))
    result = {t: v for (t, v) in zip(titles, values)}
    
    result['ID карточки'] = card_id
    
    result['Название'] = get_text(driver, 'product__title')
    result['Автор'] = get_text(driver, 'product__author')
    
    rating = get_text(driver, 'rating')
    if rating is None:
        print(card_id)
        print(card_html)
    else:
        result['Рейтинг'] = float(rating[:rating.find('(')])
        result['Голоса'] = int(rating[rating.find('(') + 1:rating.find(')')])
    
    result['Цена'] = get_text(driver, 'price')
    
    image_link = driver.find_element_by_class_name('product__image')
    if image_link is not None:
        image_link = image_link.find_element_by_tag_name('img').get_attribute('src')
    result['Обложка'] = image_link
    
    print('elapsed time: {}'.format(round(time() - start, 3)))
    
    return result

In [206]:
import pandas as pd

from time import time
start = time()

with webdriver.Chrome(executable_path='C:\\Anaconda3\\chromedriver.exe') as driver:
    result = list(map(lambda id_: extract_book_info(driver, id_), card_ids))

print('total time: {}'.format(time() - start))

1 from 441; elapsed time: 15.149
2 from 441; elapsed time: 127.966
3 from 441; elapsed time: 131.912
4 from 441; elapsed time: 135.767
5 from 441; elapsed time: 139.099
6 from 441; elapsed time: 142.335
7 from 441; elapsed time: 146.067
8 from 441; elapsed time: 149.812
9 from 441; elapsed time: 153.627
10 from 441; elapsed time: 156.839
11 from 441; elapsed time: 165.697
12 from 441; elapsed time: 169.099
13 from 441; elapsed time: 172.533
14 from 441; elapsed time: 175.969
15 from 441; elapsed time: 179.534
16 from 441; elapsed time: 183.03
17 from 441; elapsed time: 186.478
18 from 441; elapsed time: 189.647
19 from 441; elapsed time: 193.904
20 from 441; elapsed time: 197.816
21 from 441; elapsed time: 201.851
22 from 441; elapsed time: 205.612
23 from 441; elapsed time: 208.911
24 from 441; elapsed time: 213.276
25 from 441; elapsed time: 217.253
26 from 441; elapsed time: 220.735
27 from 441; elapsed time: 224.495
28 from 441; elapsed time: 227.795
29 from 441; elapsed time: 231.

231 from 441; elapsed time: 1210.263
232 from 441; elapsed time: 1213.825
233 from 441; elapsed time: 1217.034
234 from 441; elapsed time: 1220.401
235 from 441; elapsed time: 1223.777
236 from 441; elapsed time: 1256.62
237 from 441; elapsed time: 1259.968
238 from 441; elapsed time: 1263.22
239 from 441; elapsed time: 1266.662
240 from 441; elapsed time: 1270.221
241 from 441; elapsed time: 1273.785
242 from 441; elapsed time: 1277.284
243 from 441; elapsed time: 1280.764
244 from 441; elapsed time: 1284.26
245 from 441; elapsed time: 1287.641
246 from 441; elapsed time: 1291.231
247 from 441; elapsed time: 1294.596
248 from 441; elapsed time: 1298.097
249 from 441; elapsed time: 1301.942
250 from 441; elapsed time: 1305.607
251 from 441; elapsed time: 1308.966
252 from 441; elapsed time: 1312.357
253 from 441; elapsed time: 1315.666
254 from 441; elapsed time: 1318.942
255 from 441; elapsed time: 1322.383
256 from 441; elapsed time: 1325.875
257 from 441; elapsed time: 1329.304
258 

In [207]:
df = pd.DataFrame(result)
df.sort_values(by='ID карточки', inplace=True)

cols = df.columns.tolist()
cols.remove('Обложка')
cols.append('Обложка')
df = df[cols]

with open('data/hw_4.csv', mode='w', encoding='utf-8') as f_csv:
    df.to_csv(f_csv, index=False)

In [208]:
df

,ID карточки,ID товара:,ISBN,Автор,Возраст,Год издания,Голоса,Издательство,Кол-во страниц,Название,Переводчик,Редактор,Рейтинг,Серия,Тип обложки,Тираж,Формат,Художник,Цена,Обложка
0,244296,1400705,9785367009781,Фрай М.,NaN,2009,0,"Амфора, ООО",478,"Книга для таких, как я",NaN,NaN,0.0,ФРАМ,Твердая бумажная,4000,20.5 x 13.2 x 3,NaN,588 ₽,https://img-gorod.ru/upload/iblock/73b/73b0845...
1,245728,1665067,9785942783372,Фрай М.,NaN,2010,0,"Амфора, ООО",606,Волонтеры вечности,NaN,NaN,0.0,NaN,Твердая бумажная,3000,20.7 x 13.8 x 3.8,NaN,698 ₽,https://img-gorod.ru/upload/iblock/bd5/bd5917e...
250,246061,1668112,9785699110018,Емец Д.,NaN,2003,0,Эксмо-Пресс,384,Таня Гроттер и Золотая Пиявка,NaN,NaN,0.0,Таня Гроттер,Твердая бумажная,NaN,20.6 x 13.5 x 2.3,NaN,425 ₽,https://img-gorod.ru/upload/iblock/236/236a031...
2,246246,1801350,5942783535,Фрай М.,NaN,2003,0,"Амфора, ООО",350,Простые волшебные вещи,NaN,NaN,0.0,Лабиринты Эхо,Твердая бумажная,NaN,20.8 x 13.7 x 2.3,NaN,698 ₽,https://img-gorod.ru/upload/iblock/82d/82d0b11...
3,246247,1801351,9785367016239,Фрай М.,NaN,2009,0,"Амфора, ООО",367,Темная сторона,NaN,NaN,0.0,Лабиринты Ехо,Твердая бумажная,3000,20.6 x 13.5 x 2.3,NaN,698 ₽,https://img-gorod.ru/upload/iblock/a10/a1063f0...
4,246533,1804075,9785942783600,Фрай М.,12+,2013,1,"Амфора, ООО",639,Чужак,NaN,NaN,5.0,Лабиринты Ехо,Твердая бумажная,3000,20.7 x 13.6 x 4.1,NaN,698 ₽,https://img-gorod.ru/upload/iblock/2ae/2ae746b...
5,246749,1806167,9785942783877,Фрай М.,12+,2013,0,"Амфора, ООО",590,Болтливый мертвец,NaN,Стукалина Марина,0.0,Лабиринты Эхо,Твердая бумажная,2000,20.7 x 13.8 x 3.9,NaN,367 ₽,https://img-gorod.ru/upload/iblock/e40/e406f84...
6,246913,1807965,9785367003390,Фрай М.,NaN,2003,0,"Амфора, ООО",350,Власть несбывшегося,NaN,NaN,0.0,NaN,Твердая бумажная,NaN,20.6 x 13.6 x 2.3,NaN,322 ₽,https://img-gorod.ru/upload/iblock/052/052c143...
7,246914,1807966,9785942783884,Фрай М.,12+,2009,0,"Амфора, ООО",383,Наваждения,NaN,NaN,0.0,Лабиринты Ехо,Твердая бумажная,5000,20.6 x 13.7 x 2.5,NaN,427 ₽,https://img-gorod.ru/upload/iblock/6c2/6c220c8...
8,247198,1810879,9785367004069,Фрай М.,NaN,2003,0,"Амфора, ООО",557,Гнезда Химер Хроники Овётганны,NaN,NaN,0.0,NaN,Твердая бумажная,NaN,20.7 x 13.8 x 3.4,NaN,698 ₽,https://img-gorod.ru/upload/iblock/792/7923d42...
